# Data Cleaning
*Erstellt von:* Thomas Schlögl\
*Datum:* 2024/08/17

**Data Cleaning** (Datenbereinigung) bedeutet schlechte oder fehlende Daten zu *fixen*, d. h. zu korrigieren oder einzufügen um die Datenqualität zu verbessern.

Schlechte Datenqualität kann sich durch mehrere Dinge ausdrücken und hat abhängig von der *Natur der Daten* verschiedene Möglichkeiten diese zu korrigieren.

| Fehlerarten                     | Maßnahme                                                                 |
|-----------------------------|--------------------------------------------------------------------------|
| **Doppelte Daten** (Duplicates)   | Entfernen der doppelten Zeilen `drop_duplicates()` |
| **Fehlende Daten** (Missing Data) | - Interpolieren `interpolate()`<br>- Forward Fill `ffill()` Ersetzen mit dem Wert der vorherigen Zeile<br>- Backward Fill `bfill()` Ersetzen mit dem Wert der nachfolgenden Zeile<br> - Entfernen der Zeilen in denen Daten fehlen `dropna()`<br>- Ersetzen durch statistische Werte `mean()`, `median()`, `mode()` mittels `fillna()`<br> |
| **Ausreißer** (Outliers) | Erkennen und Behandeln. Oft ist der *Median* nützlich bzw. ein Median-Filter `rolling()`. |
| **Fehlerhafte Daten** (Wrong Data) | Erkennen und Korrigieren, z. B. falsche Timestamps in einem Sensorfile, die nicht kontinuierlich ansteigen. |

In [ ]:
import pandas as pd

df = pd.read_csv('data/heartrate.csv')

In [ ]:
df.info() # für den ersten Überblick

Der Non-Null Count ist bei *Date* und *Calories* ungleich 28, d. h. es fehlen Werte.

# Doppelte Daten (Duplicates)
<a id='section_duplicates'></a>
`duplicated()` liefert für jede Zeile, ob sie doppelt vorkommt, d. h. eine Series vom Typ 'bool'.

In [ ]:
# Anzahl der doppelten Zeilen
print(df.duplicated().sum())

# Gibt die doppelte Zeile aus.
df[df.duplicated()]

In [ ]:
# Gibt den Index der doppelten Zeilen aus.
df[df.duplicated()].index

In [ ]:
## Entfernen aller Duplicates

In [ ]:
# Anlegen eines neuen Dataframes ohne Duplicates
df_no_duplicates = df.drop_duplicates()
print(f'Original DataFrame: {len(df)}') # Originaldataframe
print(f'DataFrame ohne Duplikate: {len(df_no_duplicates)}') # ein Entry wurde entfernt

In [ ]:
# Ohne Anlegen eines neuen Dataframes (Inplace)
df.drop_duplicates(inplace=True)
len(df) # ein Entry wurde in df entfernt

In [ ]:
# Entfernen von Duplikaten aufgrund von bestimmten Spalten
# Beispiel: Es soll jeder Pulswert nur einmal vorkommen.
# Anmerkung: Ist bei diesen Daten sicher nicht sinnvoll, aber demonstriert die Möglichkeiten. ;-)
df = pd.read_csv('data/heartrate.csv')
df_no_pulse_duplicates = df.drop_duplicates(subset=['Pulse'])

print(df_no_pulse_duplicates)

In [ ]:
# Es läßt sich auch steuern, welche der doppelten Zeilen genommen wird.
# Es wird die letzte Zeile genommen.
df_no_pulse_duplicates_last = df.drop_duplicates(subset=['Pulse'], keep='last')

print(df_no_pulse_duplicates_last)

# Fehlende Daten (Missing Data)

In [ ]:
df = pd.read_csv('data/heartrate.csv')

# alle Zeilen in denen das Datum fehlt
nan_Dates = df[df['Date'].isna()].index
nan_Dates.tolist()

In [ ]:
# Anzeige der Zeilennummern (Indices) in denen Date fehlt
df.loc[nan_Dates.tolist()]

In [ ]:
# Anzeige der Zeilennummern (Indices) in denen Calories fehlt
nan_Calories = df[df['Calories'].isna()].index
nan_Calories.tolist()

In [ ]:
# alle Zeilen in denen Calories fehlt
df.loc[nan_Calories.tolist()]

## Interpolation von fehlenden Daten mit `interpolate()`
**Interpolation** ist ein Verfahren, bei dem fehlende Daten anhand von bekannten Daten berechnet werden. Die Interpolation kann entweder **linear**, **polynomial**, **spline** oder als Zeitreihe mit **time** erfolgen.

Hinweis: *NaN* wird in pandas mit `pd.NA` repräsentiert, in der Library numpy als `np.nan`. Die beiden Konstanten verhalten sich aber unterschiedlich, wie bei der Methode `interpolate()`.

`interpolate()` führt defaultmäßig eine lineare Interpolation durch.

Bei einer *linearen Interpolation* wird für den fehlenden Wert der arithmetische Mittelwert der davor und dahinterliegenden Datenpunkt ersetzt.

In [ ]:
# Beispiel DataFrame mit fehlenden Werten
import numpy as np

data = {
    'Sensor A': [20.3, 25.4, np.nan, 28.6, 27.2],
    'Sensor B': [20.3, np.nan, 21.3, 18.4, np.nan]
}
df = pd.DataFrame(data)
print(df)

'''
Alle NaNs werden linear interpoliert:
(25.4+28.6)/2 -> 27.0
(20.3+21.3)/2 -> 20.8
Für den letzten Wert wird der vorhergehende eingesetzt. 
'''
df_linear = df.interpolate()
df_linear

Mit dem Parameter `method` kann man auch andere mathematische Methoden auswählen, z. B. polynomial.\
Diese erfordert allerdings **scipy** einer weiteren Python Data Library.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html

## Ersetzen von fehlenden Daten mit `ffill()` und `bfill()`
Diese beiden Methoden dienen dazu alle fehlenden Daten entweder durch die vorhergehenden Daten (forward fill) oder die nachfolgenden Daten (backward fill) zu ersetzen.

Die Methode `fillna` unterstützt beide Methoden ebenso ist aber deprecated.

In [ ]:
# Beispiel DataFrame mit fehlenden Werten
import numpy as np

data = {
    'Sensor A': [20.3, 25.4, np.nan, 28.6, 27.2],
    'Sensor B': [20.3, np.nan, 21.3, 18.4, np.nan]
}
df = pd.DataFrame(data)
print(df)

# Alle NaNs werden durch den vorherigen Wert ersetze.
df_forward = df.ffill()
df_forward

In [ ]:
# Alle NaNs werden durch den nachfolgenden Wert ersetzt.
df_backward = df.bfill()

# das letzte NaN bei Sensor B kann daher nicht ersetzt werden.
df_backward

## Entfernen von Zeilen mit fehlenden Daten

### Entfernen von allen Zeilen bei denen in einer beliebigen Spalte Daten fehlen

In [ ]:
# erzeugt einen neuen Dataframe
new_df = df.dropna()
len(new_df) # -> Die Zeilen wurden entfernt, der Index bleibt aber unverändert.

In [ ]:
new_df

Mit `df.dropna(inplace=True)` wird der DataFrame `df` selbst geändert und es wird kein neuner Dataframe angelegt. Die alten Daten werden überschrieben.

In [ ]:
df.dropna(inplace=True)
df

### Entfernen von allen Zeilen bei denen in einer bestimmten Spalte Daten fehlen

In [ ]:
df = pd.read_csv('data/heartrate.csv')
new_df.dropna(subset=['Date'], inplace = True)

new_df

### Ersetzen der fehlenden Werte

#### Ersetzen aller fehlenden Werte durch einen Wert
Ist selten sinnvoll, weil auf die Datentypen der Spalten keine Rücksicht genommen wird.

In [ ]:
df = pd.read_csv('data/heartrate.csv')

# Problem: Nicht nur die fehlenden Kaloriewerte werden ersetzt, sondern auch das Datum!!!
df.fillna(444, inplace = True) 
df

#### Ersetzen aller fehlenden Einträge in einer bestimmten Spalte
*Anmerkung:* Der Code `df['Calories'].fillna(444, inplace=True)` der in einer Series den Wert umsetzt, wird ab pandas 3.0 nicht mehr unterstützt.\
`fillna()` kann ein Dictionary übergeben werden, das alle zu ersetzenden Werte mit den Spaltenbezeichnungen enthält.

In [ ]:
df = pd.read_csv('data/heartrate.csv')
df.fillna({'Calories':444}, inplace = True)
df

In [ ]:
df = pd.read_csv('data/heartrate.csv')

# Alle Calories mit *NaN* werden auf 444 geändert.
df.fillna({'Calories':444}, inplace = True)
df

In [ ]:
df = pd.read_csv('data/heartrate.csv')

# Es können in dem Dictionary auch alle Spalten angegeben werden.
df.fillna({'Calories':444, 'Date':'2023/02/22'}, inplace = True)
df

#### Ersetzen aller fehlenden Einträge mit dem Mittelwert, Median oder dem häufigsten Wert
- `mean()` berechnet den arithmetischen Mittelwert einer Zahlenserie oder Spalte.
- `median()` berechnet den Medianwert einer Zahlenserie oder Spalte.
- `mode()` berechnet den häufigsten Wert einer Zahlenserie oder Spalte.

#### Mittelwert `mean()`

In [ ]:
df = pd.read_csv('data/heartrate.csv')

mittelwert = df['Calories'].mean()
print(f'Mittelwert der Calories={mittelwert}')

In [ ]:
df.fillna({'Calories':mittelwert}, inplace = True)
df

#### Median `median()`
Der Median ist der "mittlere" Wert, wenn man alle Zahlen sortiert hat.\

In [ ]:
df = pd.read_csv('data/heartrate.csv')

median = df['Calories'].median()
print(f'Median der Calories={median}')

In [ ]:
df.fillna({'Calories': median}, inplace = True)
df

#### Häufigster Wert `mode()`

In [ ]:
df = pd.read_csv('data/heartrate.csv')

haeufigster = df['Calories'].mode()
print(f'Der häufigste Kalorienwert ist {haeufigster[0]}.')

**Mehrere häufigste Werte:** `mode()` gibt eine Series zurück, da es ja mehrere Werte geben kann, die am häufigsten vorkommen. Daher ist es sinnvoll, dann der einfachheithalber den ersten der häufigsten Werte zu verwenden.

**Beispiel mit mehreren häufigsten Werten**

In [ ]:
# Jeweils zwei Personen sind 22 und 26 Jahre alt.
people = {
    "Name" : ["Anna", "Ben", "Clara", "David", "Eva"],
    "Alter" : [24, 26, 22, 22, 26],
}

df = pd.DataFrame(people)
haeufigste = df["Alter"].mode()

# mode() liefert daher eine Series mit zwei Einträge mit dem Index 0 und 1.
print(haeufigste)

# Ausreißer (Outlier)
Plötzliche Wertausreißer, können je nach der Art der Daten durch einen Medianfilter eliminiert werden. Dabei wird der Medianwert für jedes Fenster (d.h., für jede Gruppe von drei aufeinanderfolgenden Werten) in den einzelnen Spalten berechnet.

Mit der Funktion `rolling` wird das Fenster über die Daten verschoben und dann mit `median()` für jede Fensterposition berechnet.

In [ ]:
# 100 und 80 sind potentielle Outliers, d. h. Werte die von den anderen stark abweichen.
data = {
    'A': [1, 2, 80, 4, 5, 6, 7, 100, 9, 10],
    'B': [10, 20, 15, 80, 25, 35, 40, 45, 50, 55]
}
df = pd.DataFrame(data)

# Rolling-Median mit einer Fenstergröße von 3 anwenden
df_rolling_median = df.rolling(window=3, center=True).median()

# Der erste und letzte Wert können nicht berechnet werden, da das Fenster für diese Zeilen nicht vollständig in der Datenmenge liegt.
print(df_rolling_median)

`min_periods=1` stellt sicher, dass der Median auch für die erste und letzte Zeile berechnet werden kann, wenn weniger als 3 gültige (nicht-NaN) Werte im Fenster vorhanden sind. Es wird also bereits bei einem einzigen gültigen Wert im Fenster ein Ergebnis berechnet, anstatt NaN zurückzugeben.

In [ ]:
# Rolling-Median mit einer Fenstergröße von 3 anwenden
df_rolling_median = df.rolling(window=3, center=True, min_periods=1).median()

# Der erste und letzte Wert sind jetzt ebenfalls eine Zahl.
print(df_rolling_median)

In [ ]:
df

### Zusatz: Erklärung wie der Medianwert von `median()` berechnet wird
pandas eliminiert in der `median()`-Funktion aus der Zahlenliste zunächst alle *NaN* und alle doppelten Einträge. Dann werden alle Zahlen sortiert.
- Ist die Anzahl der Zahlen ungerade, ist die mittlere Zahl der Median.
- Ist die Anzahl der Zahlen gerade, ist der Median das arithmetische Mittel zwischen den beiden mittleren Zahlen.

**Beispiel 1 mit ungerader Anzahl von Zahlen**

Bestimme den Median der Liste **[10, 1, 3, pd.NA, 2, 4, 3, 1, pd.NA]** durch überlegen.\
Überprüfe dann mittels der `median()`-Funktion.
Anmerkung: `pd.NA` ist die Konstante für *NaN*

[10, 1, 3, pd.NA, 2, 4, 3, 1, pd.NA]\
Entfernen aller NaNs -> [10, 1, 3, 2, 4, 3, 1]\
Entfernen aller Doppelten -> [10, 1, 3, 2, 4]\
Sortieren -> [1, 2, 3, 4, 10]\
**Die Länge ist 5 und ungerade.** Der Median ist daher der mittlere Wert -> 3

In [ ]:
# Überprüfung
series = pd.Series([10, 1, 3, pd.NA, 2, 4, 3, 1, pd.NA])
print(series.median())

**Beispiel 2 mit gerader Anzahl von Zahlen**

Bestimme den Median der Liste **[10, 1, 3, pd.NA, 2, 3, 1, pd.NA]** durch überlegen.\

[10, 1, 3, pd.NA, 2, 3, 1, pd.NA]\
Entfernen aller NaNs -> [10, 1, 3, 2, 3, 1]\
Entfernen aller Doppelten -> [10, 1, 3, 2]\
Sortieren -> [1, 2, 3, 10]\
**Die Länge ist 4 und gerade.** Der Median ist daher das arithmetische Mittel der beiden mittleren Werte 2 und 3 -> 2.5

In [ ]:
# Überprüfung
series = pd.Series([10, 1, 3, pd.NA, 2, 3, 1, pd.NA])
print(series.median())